In [ ]:
# pip install scikit-learn spacy requests beautifulsoup4
# opcional para pt: python -m spacy download pt_core_news_sm

import os
import re
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# tenta carregar spaCy (opcional). Se não tiver, o código funciona sem ele.
try:
    import spacy
    try:
        nlp = spacy.load("pt_core_news_sm")  # se você fala pt, baixe este modelo
    except Exception:
        # se o modelo não estiver instalado, usa pipeline em branco (sem sentencizer/lemmatizer)
        nlp = None
except Exception:
    nlp = None

# ----------------------------
# helpers: coletar e extrair texto
# ----------------------------
def fetch_url_text(url):
    """Baixa uma página e retorna o texto dos parágrafos."""
    r = requests.get(url, timeout=10)
    soup = BeautifulSoup(r.text, "html.parser")
    ps = soup.find_all("p")
    text = "\n".join(p.get_text() for p in ps)
    return text

def load_texts_from_folder(folder):
    """Carrega .txt de uma pasta e retorna lista de strings."""
    texts = []
    for filename in os.listdir(folder):
        if filename.endswith(".txt"):
            with open(os.path.join(folder, filename), "r", encoding="utf-8") as f:
                texts.append(f.read())
    return texts

# ----------------------------
# pré-processamento e split em sentenças
# ----------------------------
def split_sentences(text):
    """Tenta usar spaCy para sentenças; se não, faz split simples."""
    if nlp:
        doc = nlp(text)
        return [sent.text.strip() for sent in doc.sents if sent.text.strip()]
    # fallback simples (bom o suficiente pra protótipo)
    sents = re.split(r'(?<=[.!?])\s+', text)
    return [s.strip() for s in sents if s.strip()]

def preprocess_sentence(sent):
    """Limpeza leve; se spaCy estiver disponível, lemmatiza e remove stopwords."""
    if nlp:
        doc = nlp(sent)
        toks = [t.lemma_.lower() for t in doc if not t.is_stop and not t.is_punct]
        return " ".join(toks)
    # fallback simples
    sent = sent.lower()
    sent = re.sub(r'\s+', ' ', sent)
    return sent.strip()

# ----------------------------
# construir índice TF-IDF
# ----------------------------
class SimpleQAIndex:
    def __init__(self):
        self.sentences = []
        self.sent_orig = []
        self.vectorizer = TfidfVectorizer()
        self.matrix = None

    def add_text(self, text):
        sents = split_sentences(text)
        for s in sents:
            proc = preprocess_sentence(s)
            if proc:
                self.sentences.append(proc)
                self.sent_orig.append(s)

    def build(self):
        if not self.sentences:
            raise ValueError("Índice vazio. Adicione textos primeiro.")
        self.matrix = self.vectorizer.fit_transform(self.sentences)

    def answer(self, question, top_k=3):
        q = preprocess_sentence(question)
        qv = self.vectorizer.transform([q])
        sims = cosine_similarity(qv, self.matrix)[0]
        idxs = np.argsort(sims)[::-1][:top_k]
        results = []
        for i in idxs:
            results.append({"score": float(sims[i]), "sentence": self.sent_orig[i]})
        return results

# ----------------------------
# Exemplo de uso
# ----------------------------
if __name__ == "__main__":
    qa = SimpleQAIndex()

    # 1) adicionar textos locais
    # texts = load_texts_from_folder("meus_textos")
    # for t in texts:
    #     qa.add_text(t)

    # 2) ou baixar uma URL (exemplo)
    exemplo_url = "https://pt.wikipedia.org/wiki/Pizza"  # só pra teste
    txt = fetch_url_text(exemplo_url)
    qa.add_text(txt)

    # 3) construir o índice (TF-IDF)
    qa.build()

    # 4) testar perguntas
    pergunta = "Qual a origem da pizza?"
    respostas = qa.answer(pergunta, top_k=2)
    for r in respostas:
        print(f"[{r['score']:.3f}] {r['sentence']}")
